In [4]:
import numpy as np
import pandas as pd

In [5]:
def MV_criterion(weights, df):
    """
    Ountput:
        optimización del criterio (media-varianza) de la cartera
    Inputs:
        weights(type ndarray numpy): Pesos de la cartera
        df(type ndarray numpy): Retornos de los activos
    """
    # Parámetros
    Lambda = 0
    W = 1
    Wbar = 1 + 0.25/100
    # Calcular la rentabilidad de la cartera
    portfolio_return = np.multiply(df, np.transpose(weights))
    portfolio_return = portfolio_return.sum(axis=1)
    # Calcular la media y la volatilidad de la cartera
    mean = np.mean(portfolio_return, axis=0)
    std = np.std(portfolio_return, axis=0)
    # Calcular el criterio
    criterion = (Wbar**(1-Lambda)/(1+Lambda)+
                 Wbar**(-Lambda)*W*mean-
                 Lambda/2*Wbar**(-1-Lambda)*
                 W**2*std**2
                 )
    return -criterion

In [6]:
def SK_criterion(weights, df):
    """ 
    Output: 
        optimization porfolio criterion                                   
    Inputs: 
        weight (type ndarray numpy): Wheight for portfolio               
        data (type ndarray numpy): Returns of stocks                     
    """
    # Parámetros
    Lambda = 0
    W = 1
    Wbar = 1 + 0.25 / 100
    # Calcular la rentabilidad de la cartera
    portfolio_return = np.multiply(df, np.transpose(weights))
    portfolio_return = portfolio_return.sum(axis=1)
    # # Calcular la media, volatilidad, skew y la kurtosis de la cartera
    mean = np.mean(portfolio_return, axis=0)
    std = np.std(portfolio_return, axis=0)
    skewness = skew(portfolio_return, 0)
    kurt = kurtosis(portfolio_return, 0)
    # Calcular el criterio
    criterion = Wbar ** (1 - Lambda) / (1 + Lambda) + Wbar ** (-Lambda) \
    * W * mean - Lambda / 2 * Wbar ** (-1 - Lambda) * W ** 2 * std ** 2 \
    + Lambda * (Lambda + 1) / (6) * Wbar ** (-2 - Lambda) * W ** 3 * skewness \
    - Lambda * (Lambda + 1) * (Lambda + 2) / (24) * Wbar ** (-3 - Lambda) *\
     W ** 4 * kurt
    
    return -criterion

In [7]:
def SR_criterion(weights, df):
    """
    Ountput:
        optimización del criterio (ratio de Sharpe) de la cartera
    Inputs:
        weights(type ndarray numpy): Pesos de la cartera
        df(type ndarray numpy): Retornos de los activos
    """
    # Calcular la rentabilidad de la cartera
    portfolio_return = np.multiply(df, np.transpose(weights))
    portfolio_return = portfolio_return.sum(axis=1)
    # Calcular la media y la volatilidad de la cartera
    mean = np.mean(portfolio_return, axis=0)
    std=np.std(portfolio_return, axis=0)
    # Calcular el opuesto al ratio de Sharpe
    sharpe = mean / std
    return -sharpe

In [8]:
def SOR_criterion(weights, df):
    """
    Ountput:
        optimización del criterio (ratio de Sortino) de la cartera
    Inputs:
        weights(type ndarray numpy): Pesos de la cartera
        df(type ndarray numpy): Retornos de los activos
    """
    # Calcular la rentabilidad de la cartera
    portfolio_return = np.multiply(df, np.transpose(weights))
    portfolio_return = portfolio_return.sum(axis=1)
    # Calcular la media y la volatilidad de la cartera
    mean = np.mean(portfolio_return, axis=0)
    std = np.std(portfolio_return[portfolio_return < 0], axis=0)
    # Calcular el opuesto al ratio de Sortino
    Sortino = mean / std
    return -Sortino

In [1]:
def beta_function(portfolio, benchmark):
    """
    Ouput:
        Métrica Beta CAPM
    Inputs:
        portfolio (type dataframe pandas): Retornos de la cartera
        benchmark (type dataframe pandas): Retornos del índice de referencia
    """
    # Concatenar el portfolio y el índice
    join = pd.concat((portfolio, benchmark), axis=1).dropna()
    # Covarianza y varianza
    cov, var = np.cov(join, rowvar=False)[1]
    return cov/var

In [ ]:
def alpha_function(portfolio, benchmark, timeframe=252):
    """
    Ouput:
        Métrica Alpha CAPM
    Inputs:
        portfolio (type dataframe pandas): Retornos de la cartera
        benchmark (type dataframe pandas): Retornos del índice de referencia
    """
    # Calcula la beta
    beta = beta_function(portfolio, benchmark)
    mean_stock_return = portfolio.values.mean() * timeframe
    mean_market_return = benchmark.values.mean() * timeframe
    return mean_stock_return - beta * mean_market_return

In [ ]:
def sharpe_function(portfolio, timeframe=252):
    """
    Output:
        Métrica del ratio de Sharpe
    Inputs:
        portfolio (type dataframe pandas): Retornos de la cartera
        timeframe (type int): Factor de anualización
    """
    # Media y desviación típica de los retornos anualizados 
    portfolio = portfolio.values
    mean = portfolio.mean() * timeframe
    std = portfolio.std() * np.sqrt(timeframe)
    return mean / std

In [ ]:
def sortino_function(portfolio, timeframe=252):
    """
    Output:
        Métrica del ratio de Sharpe
    Inputs:
        portfolio (type dataframe pandas): Retornos de la cartera
        timeframe (type int): Factor de anualización
    """
    # Tomar valores a la baja
    portfolio = portfolio.values
    downward = portfolio[portfolio < 0]
    mean = portfolio.mean() * timeframe
    std = downward.std() * np.sqrt(timeframe)
    return mean / std

In [ ]:
def drawdown_function(portfolio):
    """
    Output: 
        Drawdown
    Inputs:
        portfolio (type dataframe pandas): Retornos de la cartera
    """
    # Calcular el producto acumulado de los rendimientos
    cum_rets = (portfolio + 1).cumprod()
    # Calcular el máximo producto acumulado
    running_max = np.maximum.accumulate(cum_rets.dropna())
    running_max[running_max < 1] = 1
    # Calcular el drawdown
    drawdown = (cum_rets / running_max - 1)
    return drawdown

In [ ]:
def VaR_function(theta, mu, sigma):
    """
    Output:
        VaR
    Inputs:
        theta (type float): umbral del error en %
        mu (type float): retorno esperado de la cartera
        sigma (type float): volatilidad de la cartera
    """
    # Número de simulaciones
    n = 100000
    # Encontrar los valores para el umbral de error theta
    t = int(n * theta)
    # Crear un vector con n simulaciones de la ley normal
    vec = pd.DataFrame(np.random.normal(mu, sigma, size=(n,)),
                       columns=['Simulations'])
    # Ordena los valores y encuentra el valor theta%
    var = vec.sort_values(by="Simulations").iloc[t].values[0]
    return var

In [ ]:
def cVaR_function(theta, mu, sigma):
    """
    Output:
        cVaR
    Inputs:
        theta (type float): umbral del error en %
        mu (type float): retorno esperado de la cartera
        sigma (type float): volatilidad de la cartera
    """
    # Número de simulaciones
    n = 100000
    # Encontrar los valores para el umbral de error theta
    t = int(n * theta)
    # Crear un vector con n simulaciones de la ley normal
    vec = pd.DataFrame(np.random.normal(mu, sigma, size=(n,)),
                       columns=['Simulations'])
    # Ordena los valores y encuentra el valor theta%
    cvar = vec.sort_values(by="Simulations").iloc[0:t, :].mean().values[0]
    return cvar

In [ ]:
def CR_function(weights, portfolio, benchmark):
    """
    Output:
        Métrica de la contribución al riesgo
    Input:
        weights (type pandas.DataFrame): Pesos de la cartera
        portfolio (type pandas.DataFrame): Retornos de los activos
        benchmark (type pandas.DataFrame): Retornos del índice de referencia
    """
    # Encontrar el número de activos en la cartera
    l = len(weights)
    # Calcular la contribución al riesgo de cada activo
    crs = []
    for i in range(l):
        cr = beta_function(portfolio.iloc[:, i], benchmark) * weights.iloc[i]
        crs.append(cr)
    # Normalizar por la suma de la contribución al riesgo
    return crs / np.sum(crs)